In [ ]:
!wget -c https://datasets.cvc.uab.es/rrc/DocVQA/Task1/spdocvqa_images.tar.gz --no-check-certificate

In [ ]:
!wget https://datasets.cvc.uab.es/rrc/DocVQA/Task1/spdocvqa_qas.zip --no-check-certificate

In [ ]:
!mkdir spdocvqa_images && tar -xzf spdocvqa_images.tar.gz -C spdocvqa_images

In [ ]:
!unzip spdocvqa_qas.zip

In [1]:
import io
import PIL
import json
from datasets import Dataset, Image
from glob import glob

In [2]:
splits = [
    'train',
    'validation',
    'test',
]
label_files = [
    'train_v1.0_withQT.json',
    'val_v1.0_withQT.json',
    'test_v1.0.json',
]
docvqa = []
for i_split, split in enumerate(splits):
    path = []
    question = []
    questionId = []
    answer = []
    with open(label_files[i_split], 'r') as f:
        split_info = json.load(f)
    data = split_info['data']
    for sample in data:
        path.append("spdocvqa_images/" + sample['image'].split("/")[1])
        question.append(sample['question'])
        questionId.append(sample['questionId'])
        if i_split == 2:
            answer.append(["Test server is available at rrc.cvc.uab.es"])
        else:
            answer.append(sample['answers'])
    docvqa.append(Dataset.from_dict({
        'path':path,
        'question':question,
        'questionId':questionId,
        'answer':answer
    }))

In [3]:
def load_image(sample):
    with open(sample['path'], 'rb') as f:
        buff = io.BytesIO(f.read()).getbuffer()
    return {
        'image': buff
    }

In [ ]:
for i_split, split in enumerate(splits):
    ds = docvqa[i_split].map(load_image, writer_batch_size=64, cache_file_name=f"{split}.cache")
    ds = ds.cast_column('image', Image())
    ds.push_to_hub("danjacobellis/docvqa", split=split)

Map:   0%|          | 0/39463 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/57 [00:00<?, ?it/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]